In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
def load_housing_csv(p):
    if Path(p).exists():
        print(f"[INFO] Loaded: {p}")
        return pd.read_csv(p)

    # 폴백: /kaggle/input 전체에서 housing.csv 탐색
    hits = list(Path("/kaggle/input").rglob(p))
    if hits:
        chosen = str(hits[0])
        print(f"[INFO] Loaded (auto-found): {chosen}")
        return pd.read_csv(chosen)

    raise FileNotFoundError(
        "요청하신 데이터셋을 /kaggle/input 경로에서 찾지 못했습니다. "
        "'Add data'에서 데이터셋 추가 후 정확한 경로를 candidates에 넣어주세요."
    )

In [ ]:
df = load_housing_csv("/kaggle/input/california-housing-prices/housing.csv")  # 또는 네 파일명

In [ ]:
if "ocean_proximity" in df.columns:
    df["ocean_proximity"] = df["ocean_proximity"].astype("category")
print(df.shape)
df.head()

In [ ]:
REQUIRED_COLS = [
    "longitude", "latitude", "housing_median_age",
    "total_rooms", "total_bedrooms", "population",
    "households", "median_income", "median_house_value",
    "ocean_proximity"
]
missing = [c for c in REQUIRED_COLS if c not in df.columns]
assert not missing, f"다음 컬럼이 없습니다: {missing}"

In [ ]:
def make_features(_df: pd.DataFrame) -> pd.DataFrame:
    d = _df.copy()
    eps = 1e-6
    d["rooms_per_household"]      = d["total_rooms"] / (d["households"] + eps)
    d["bedrooms_per_room"]        = d["total_bedrooms"] / (d["total_rooms"] + eps)
    d["population_per_household"] = d["population"] / (d["households"] + eps)
    d["lat_lon"]                  = d["latitude"] * d["longitude"]
    return d

In [ ]:
print(df.shape)
df.head()

In [ ]:
data = df.dropna(subset=["median_house_value"]).reset_index(drop=True)
print(data.shape)
data.head()

In [ ]:
from sklearn.cluster import KMeans

coords = data[["longitude", "latitude"]].values

k = 5  # 군집 개수 (보통 5~10 사이 실험)
kmeans = KMeans(n_clusters=k, random_state=42)
data["geo_cluster"] = kmeans.fit_predict(coords)

# ===== 새로운 feature 추가 결과 =====
data.groupby("geo_cluster").head(5)

data["geo_cluster"] = data["geo_cluster"].astype("category")

In [ ]:
data = make_features(data)

print(data.shape)
data.head()

In [ ]:
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train.head()

In [ ]:
num_cols = [c for c in X_train.columns if str(X_train[c].dtype) != "category" and X_train[c].dtype != "object"]
cat_cols = [c for c in X_train.columns if c not in num_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ],
    remainder="passthrough"
)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmses = []
best_n_estimators = []

for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train), 1):
    X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    y_tr, y_va = y_train.iloc[tr_idx], y_train.iloc[va_idx]

   
    preprocessor.fit(X_tr)
    X_tr_enc = preprocessor.transform(X_tr)
    X_va_enc = preprocessor.transform(X_va)
    
    # 모델 (L2 정규화 추가)
    cb = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=6,
        subsample=0.8,
        colsample_bylevel=0.8,
        random_state=42 + fold,
        thread_count=-1,
        verbose=False
    )

    # early_stopping: 검증셋은 같은 인코더로 변환된 X_va_enc 사용
    cb.fit(
        X_tr_enc, y_tr,
        eval_set=[(X_va_enc, y_va)],
        early_stopping_rounds=100,
        verbose=False
    )

    # 예측 및 RMSE
    y_pred_va = cb.predict(X_va_enc)
    rmse = mean_squared_error(y_va, y_pred_va, squared=False)
    fold_rmses.append(rmse)

    # best_iteration 안전 추출
    best_iter = cb.get_best_iteration()
    if best_iter is None:
        best_iter = cb.tree_count_
    best_n_estimators.append(int(best_iter))

    print(f"[Fold {fold}] RMSE: {rmse:,.2f} | best_n_estimators ≈ {best_iter}")

print("\nCV RMSE (mean ± std): "

      f"{np.mean(fold_rmses):,.2f} ± {np.std(fold_rmses):,.2f}")
print("Per-fold best_n_estimators:", best_n_estimators)

In [ ]:
# CV 결과에서 합리적인 트리 수 선택 (중앙값)
final_n_estimators = int(np.median(best_n_estimators)) if len(best_n_estimators) > 0 else 800
print("Final n_estimators (median of CV):", final_n_estimators)

# 전체 TRAIN으로 전처리기를 fit → 같은 변환을 TEST에도 적용
final_preprocessor = ColumnTransformer(
    [("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
    remainder="passthrough"
)
final_preprocessor.fit(X_train)

# v3 -> v4 변경
y_train_raw = y_train.copy()
y_test_raw  = y_test.copy()

# 로그 변환 (log1p = log(1 + x), 0값에도 안전)
y_train = np.log1p(y_train_raw)
y_test  = np.log1p(y_test_raw)

X_train_enc = final_preprocessor.transform(X_train)
X_test_enc  = final_preprocessor.transform(X_test)
print(y_train.head())

    
final_model = CatBoostRegressor(
    iterations=final_n_estimators,
    learning_rate=0.03,
    depth=6,
    subsample=0.8,
    colsample_bylevel=0.8,
    random_state=42,
    thread_count=-1,
    verbose=False
)

# 전체 TRAIN으로 재학습 
final_model.fit(
    X_train_enc, y_train
)

# 독립 TEST 평가
y_pred_test = final_model.predict(X_test_enc)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
r2_test = r2_score(y_test, y_pred_test)


print(f"[TEST] RMSE: {rmse_test:,.2f}")
print(f"[TEST] R²:   {r2_test:.3f}")